<br>

# 外界気象データを扱うモジュール  

本モジュールの概略を説明する。  
本モジュールは外気気象データを取り扱うモジュールである。  

外界気象は室内空間（Room）や壁体（Wall）、開口（Opening）などのモジュールとはやや異なる性質を有する。  
外界気象は境界条件として測定値を用いるため、測定値をデータセットして入力し、各時刻に応じた値を計算する必要がある。  

In [1]:
using CSV
using DataFrames
using Dates
using StringEncodings
include("air.jl")
include("solar_radiation.jl")
include("wall.jl")
include("./function/vapour.jl")

convertAH2RH (generic function with 1 method)

## 1. 外界気象を定義する構造体  

外界気象を定義する構造体Climateの説明を行う。  
ここでは入力条件として外界気象の情報を以下のように与えるものとした。

In [2]:
# 気象の構造
Base.@kwdef mutable struct Climate
    date::DateTime  = DateTime( 2000, 1, 1, 0, 0, 0 )           # 日時
    location::Dict{String, Any} = Dict( "city"  =>  "Kyoto",    # 都市名
                                        "lon"   =>  135.678,    # 緯度
                                        "phi"   =>  34.983,     # 経度
                                        "lons"  =>  135.0)      # 地方標準時の地点の経度
    air::Air        = Air(name = "climate")     # 温度・湿度等の情報
    Jp::Float64     = 0.0       # 降水量[mm/h]
    Js::Float64     = 0.0       # 積雪量[mm/h]
    WS::Float64     = 0.0       # 風速[m/s]
    WD::String      = "東"       # 風向（N,NNE,NE,ENE,E,ESE,SE,SSE,S,SSW,SW,WSW,W,WNW,NW,NNW）
    tau::Float64    = 0.0       # 大気透過率[-]
    solar::Dict{String, Float64}    = Dict( "SOLDN" => 0.0,
                                            "SOLSN" => 0.0,
                                            "LATI"  => 0.0,
                                            "ALTI"  => 0.0)
    rho::Float64    = 0.0       # 地表面日射反射率[-]
    cloudiness::Int = 0         # 雲量[-]
    input_data::DataFrame   = DataFrame()   # 入力する外気気象データ
    input_data_type::Array{Symbol,1}   = [] # 入力する外気気象データの種類
    logging_interval::Int   = 0             # 外気気象データの測定間隔
end

Climate

In [3]:
#### 方位に関する定義
direction = Dict(   "N" => 0.0, "NNE" => 22.5, "NE" => 45.0, "ENE" => 67.5, 
                    "E" => 90.0, "ESE" => 112.5, "SE" => 135.0, "SSE" => 157.5, 
                    "S" => 180.0, "SSW" => 202.5, "SW" => 225.0, "WSW" => 247.5, 
                    "W" => 270.0, "WNW" => 292.5, "NW" => 315.0, "NNW" => 337.5)
                    
direction_JP = Dict("北" => 0.0, "北北東" => 22.5, "北東" => 45.0, "東北東" => 67.5, 
                    "東" => 90.0, "東南東" => 112.5, "南東" => 135.0, "南南東" => 157.5, 
                    "南" => 180.0, "南南西" => 202.5, "南西" => 225.0, "西南西" => 247.5, 
                    "西" => 270.0, "西北西" => 292.5, "北西" => 315.0, "北北西" => 337.5,
                    "静穏" => 0.0)

cal_direction( direction_name::String ) = direction[direction_name]

cal_direction (generic function with 1 method)

In [4]:
########################################
# 出力用の関数
date(climate::Climate)  = climate.date
location(climate::Climate) = climate.location
air(climate::Climate)   = climate.air
temp(climate::Climate)  = temp(climate.air)
rh(climate::Climate)    = rh(climate.air)
ah(climate::Climate)    = ah(climate.air)
pv(climate::Climate)    = pv(climate.air)
p_atm(climate::Climate) = p_atm(climate.air)
Jp(climate::Climate)    = climate.Jp
Js(climate::Climate)    = climate.Js
WS(climate::Climate)    = climate.WS
WD(climate::Climate)    = climate.WD
θwd(climate::Climate)   = direction_JP[WD(climate)]
tau(climate::Climate)   = climate.tau
solar(climate::Climate) = climate.solar
rho(climate::Climate)   = climate.rho
cloudiness(climate::Climate)        = climate.cloudiness
input_data(climate::Climate)        = climate.input_data
input_data_type(climate::Climate)   = climate.input_data_type
logging_interval(climate::Climate)  = climate.logging_interval

########################################
# 入力用の関数
function set_date(climate::Climate, date::DateTime) 
    climate.date        = date  end
function set_location(climate::Climate, location::Dict{String, Any}) 
    climate.location    = location  end
function set_air(climate::Climate, air::Air) 
    climate.air         = air  end
function set_temp(climate::Climate, temp::Float64) 
    climate.air.temp    = temp  end
function set_rh(climate::Climate, rh::Float64) 
    climate.air.rh      = rh  end
function set_pv(climate::Climate, pv::Float64) 
    climate.air.pv      = pv  end
function set_ah(climate::Climate, ah::Float64) 
    climate.air.ah      = ah  end

########################################
# 時間経過に関する関数
function time_elapses(climate::Climate, dt::Float64)
    climate.date    = climate.date + Millisecond(dt*1000)   end

time_elapses (generic function with 1 method)

## 2. 気象データの入力
### ― 気象庁の測定データをもとに －

外界気象データは、主に以下の3つの経路で入手することが一般的である。  
- 気象ステーション等による実測  
- 気象庁のデータを活用
- 設計用気象データの活用  

以下では気象庁において測定されているデータを参考にデータの読み込み方法を定めた。  

気象庁 - 過去の気象データ検索  
http://www.data.jma.go.jp/obd/stats/etrn/index.php

気象庁 - 過去の気象データ・ダウンロード  
http://www.data.jma.go.jp/gmd/risk/obsdl/index.php


これらの入力データは"./input_data"内に格納しておく。  
このファイルの読み込み方法として下記の関数を定義しておこう。  
例えば、climate_data.csvというファイルを格納しているのであれば、  
本関数の引数はfile_name = "climate_data.csv"となる。  

### ※ヘッダーの位置はまちまちなので、入力する環境条件ごとにheaderの数字を変更すること

In [5]:
# 時間経過に関する関数
function time_elapses(climate::Climate, dt::Float64)
    climate.date    = climate.date + Millisecond(dt*1000)   end

time_elapses (generic function with 1 method)

In [6]:
# データのロギング間隔を取得する関数
function cal_logging_interval_min( data::DataFrame )
    return Int(( hour(data.date[2]) - hour(data.date[1]) ) * 60 + ( minute(data.date[2]) - minute(data.date[1]) ))
end

cal_logging_interval_min (generic function with 1 method)

In [7]:
# 気象データをinputする関数
function input_climate_data(file_name::String, header::Int = 3)

    ###################################
    # 入力ファイルの読み込み
    # 相対パスを入力の上指定さえれている場合、
    if contains(file_name, "./")
        file_directory = file_name
        
    # ファイル名＋csvの形で書かれている場合、
    elseif contains(file_name, ".csv")
        file_directory = "./input_data/climate_data/"*string(file_name)
        
    # ファイル名のみが書かれている場合、
    else
        file_directory = "./input_data/climate_data/"*string(file_name)*".csv"        
    end
    
    ###################################
    # 入力ファイルの読み込み：日本語対応
    input_data = CSV.File(open(file_directory, enc"Shift_JIS"), header = header) |> DataFrame
    # ⇒ DateTime型への変換
    input_data.date = DateTime.( input_data[!,1], dateformat"y/m/d HH:MM")
    ###################################

    # 入力するデータのタイプの確認
    data_type = []
    for names = names( input_data )
        if input_data[!,names] == 1
            # println( names, " is same")
        elseif all( ismissing, input_data[!,names])
            # println( names, " is missing")
        elseif names == "date"

        else
            # println( names, " is not same")
            push!(data_type, names)
        end
    end
    push!(data_type, :pv) # 水蒸気圧についても更新
    push!(data_type, :ah) # 絶対湿度についても更新

    # 水蒸気圧・絶対湿度の算出
    input_data.pv = [ convertRH2Pv( temp = input_data.temp[i] + 273.15, rh = input_data.rh[i]/100 ) for i = 1 : length(input_data.temp) ]
    input_data.ah = [ convertRH2AH( temp = input_data.temp[i] + 273.15, rh = input_data.rh[i]/100 ) for i = 1 : length(input_data.temp) ]

    # データのロギングインターバルの変更
    logging_interval = cal_logging_interval_min( input_data )

    ##################################
    # 気象構造の設定
    climate = Climate(
        date = input_data.date[1] ,
        input_data = input_data, 
        input_data_type = Symbol.(data_type),
        logging_interval = logging_interval)

    reset_climate_data( climate )

    return climate
end

input_climate_data(; file_name::String, header::Int = 3) = input_climate_data(file_name, header)

input_climate_data (generic function with 3 methods)


### Appendix. DataFramesを用いた値の抽出

DataFramesにおけるデータの抽出方法について示しておく。  
入力する環境データは、時刻と紐づけれるよう年月日時分秒の情報も入力しておく。

DataFramesは[]内に条件を指定することで条件抽出が可能となるため、以下のような関数を用いることである時刻における値を抽出することが出来る。

参考：Juliaで遊んでみた - データフレーム操作編
https://qiita.com/HiroyukiTachikawa/items/e01917ade931031ec6a1#4%E3%83%87%E3%83%BC%E3%82%BF%E6%8A%BD%E5%87%BA

In [8]:
# ある時刻におけるすべての列を取得する方法
function get_row_at_date( climate_data::DataFrame, date::DateTime )
    return climate_data[ climate_data[!, :date] .== date, : ]
end

# 指定した列の値を取得する方法
function get_value_by_column_name( data::DataFrame, name::String )
    return data[!, name]
end

# 以下現状（2024/11/21）使わない関数群
# 温度の抽出
function extract_temp_by_climate_data( climate_data::DataFrame, date::DateTime )
    return climate_data.temp[climate_data.date.==date,:][1] #[1]はmatrixをスカラーに変換するため
end

# 相対湿度の抽出
function extract_rh_by_climate_data( climate_data::DataFrame, date::DateTime )
    return climate_data.rh[climate_data.date.==date,:][1]
end

# 水蒸気圧の抽出
function extract_pv_by_climate_data( climate_data::DataFrame, date::DateTime )
    return climate_data.pv[climate_data.date.==date,:][1]
end

extract_pv_by_climate_data (generic function with 1 method)

## 3. 環境データの直線内挿（線形補間）  

環境データは、数値解析の時間刻みに合うように測定間隔ごとのデータを補間（内挿）する必要がある。  
以下では、一般的な補間方法である線形補間（Linear Interpolate(Lerp)）方法について示す。 


### 3.1 線形補完の基礎方程式

線形補完の基本的な概念を下の図に示す。  
t1およびt2の値が既知であるとき、ti時における値yiは以下の関係で結ばれる。


<img src="picture/線形内挿.png" width="40%">


$$\frac{ y_i - y_1 }{ y_2 - y_1 } = \frac{ t_i - t_1 }{ t_2 - t_1 }$$  

すなわち、ある時刻$t_i$における値$y_i$は以下のように求められる。

$$ y_i = y_1 + (y_2 - y_1)\frac{ t_i - t_1 }{ t_2 - t_1 }$$

In [9]:
function cal_lerp(; y1::Float64, y2::Float64, t1::Float64, t2::Float64, ti::Float64 )
    return y1 + ( y2 - y1 ) * ( ti - t1 ) / ( t2 - t1 )
end

cal_lerp (generic function with 1 method)

### 3.2 環境データの線形内挿  

上記の関数を環境データで置き換えた場合、各変数は下記のように定義できる。
- interval($t_2- t_1$)：データの測定間隔
- remainder($t_i- t_1$)：現在時刻を測定間隔で割ったものの余り


In [10]:
function cal_lerp(; y1::Float64, y2::Float64, interval::Float64, remainder::Float64 )
    return y1 + ( y2 - y1 ) * ( remainder ) / ( interval )
end

cal_lerp (generic function with 1 method)

## 4. ある時刻における温度・湿度を求める関数

### 4.1 直前の測定データの日時を求める関数  
まず、ある時刻における値を求めるには、その直前のロギングインターバル時($t_i$)を求める必要がある。これを以下のように関数化する。

※logging_interval_minは分で入力する

In [11]:
function cal_before_date( date::DateTime, logging_interval_min::Int )
    min   =  
        if logging_interval_min == 60
            0 
        else 
            minute(date) - mod(minute(date), logging_interval_min)
        end
    return DateTime( year(date), month(date), day(date), hour(date), min, 0 )
end

cal_before_date (generic function with 1 method)

In [12]:
function cal_after_date( date_before::DateTime, logging_interval_min::Int )
    return date_before + Minute(logging_interval_min)   
end

cal_after_date (generic function with 1 method)

### 4.2 ある時刻における値を求める関数

※2024/11/22時点　削除予定

#### 入力引数  
- climate_data：環境データ（DataFrames）
- date：現在の時刻（Calendar）
- logging_interval_min：分数標記のロギング間隔（通常環境データを取得する際に得られる）  

#### 出力値  
- temp, pv, rhの配列

In [13]:
function cal_temp_rh_pv_by_climate_data( climate::Climate, date::DateTime )
    
    # 直前の測定データの日時を求める
    date_before = cal_before_date( date, climate.logging_interval )
    
    # 直後の測定データの日時を求める
    date_after  = cal_after_date( date_before, climate.logging_interval )
    
    # 直線内挿による現在の値の入力
    temp = cal_lerp(
        y1 = extract_temp_by_climate_data( climate.data, date_before ), 
        y2 = extract_temp_by_climate_data( climate.data, date_after ),
        interval  = float(climate.logging_interval),
        remainder = float(mod(minute(date), climate.logging_interval) ))
    
    pv = cal_lerp(
        y1 = extract_pv_by_climate_data( climate.data, date_before ), 
        y2 = extract_pv_by_climate_data( climate.data, date_after ),
        interval  = float(climate.logging_interval),
        remainder = float(mod(minute(date), climate.logging_interval) ))
    
    rh = convertPv2RH( temp = temp + 273.15 , pv = pv )
    return temp, pv, rh
end

cal_temp_rh_pv_by_climate_data (generic function with 1 method)

### 4.3 Climate構造体中の値を新値で更新する

In [14]:
function reset_climate_data( climate::Climate )
    
    # 直前・直後の測定データの日時を求める
    date_before = cal_before_date( climate.date, climate.logging_interval )
    date_after  = cal_after_date( date_before, climate.logging_interval )

    # 直前・直後のデータを取得
    data_before = get_row_at_date( climate.input_data, date_before )
    data_after  = get_row_at_date( climate.input_data, date_after )

    # 現在値を書き換える（オプション情報）
    for field_name = climate.input_data_type
        # temp, rh, p_atmはair構造体の中の変数
        if field_name == :temp
            setfield!( climate.air, field_name, cal_lerp(
                y1 = float(data_before[!,field_name][1]), 
                y2 = float(data_after[!,field_name][1]),
                interval  = float(climate.logging_interval),
                remainder = float(mod(minute(climate.date), climate.logging_interval))) + 273.15)
        elseif field_name == :rh || field_name == :p_atm || field_name == :pv || field_name == :ah
            setfield!( climate.air, field_name, cal_lerp(
                y1 = float(data_before[!,field_name][1]), 
                y2 = float(data_after[!,field_name][1]),
                interval  = float(climate.logging_interval),
                remainder = float(mod(minute(climate.date), climate.logging_interval))))
        # その他はclimate構造体の中の変数
        elseif field_name == :WD
            setfield!( climate, field_name, String(data_before[!,field_name][1]) )         
        else
            setfield!( climate, field_name, cal_lerp(
                y1 = float(data_before[!,field_name][1]), 
                y2 = float(data_after[!,field_name][1]),
                interval  = float(climate.logging_interval),
                remainder = float(mod(minute(climate.date), climate.logging_interval))))
            # 大気透過率を更新する場合、日射量の計算を行う
            if field_name == :tau
                climate.solar["SOLDN"], climate.solar["SOLSN"], climate.solar["LATI"], climate.solar["ALTI"] = 
                    cal_solar_radiation( 
                        year = year(climate.date), month = month(climate.date), day = day(climate.date), 
                        hour = hour(climate.date), min = minute(climate.date), sec = second(climate.date), 
                        PHI = climate.location["phi"], LON = climate.location["lon"], LONS = climate.location["lons"], 
                        P   = climate.tau )
            end
        end
    end
    # 相対湿度に関しては水蒸気圧から再度計算しなおす
    setfield!( climate.air, :rh, convertPv2RH( temp = climate.air.temp , pv = climate.air.pv ) )
end

reset_climate_data (generic function with 1 method)

In [16]:
# 検証
test = input_climate_data( file_name = "../input_data/climate_data/sample_climate_data_test.csv", header = 3)


Climate(DateTime("2012-01-01T00:00:00"), Dict{String, Any}("city" => "Kyoto", "phi" => 34.983, "lon" => 135.678, "lons" => 135.0), Air(0, "climate", 0.0, 0.0, 0.0, 0.0, 296.15, 0.75, 2107.83143992347, 0.013214156124990986, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0, 0.0, 0.0, "E", 0.0, Dict("SOLDN" => 0.0, "LATI" => 0.0, "SOLSN" => 0.0, "ALTI" => 0.0), 0.0, 0, 1441×13 DataFrame
  Row │ date                 temp   rh     p_atm    Jp       Js       WS       ⋯
      │ DateTime             Int64  Int64  Missing  Missing  Missing  Missing  ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ 2012-01-01T00:00:00     23     75  missing  missing  missing  missing  ⋯
    2 │ 2012-01-01T00:05:00     23     75  missing  missing  missing  missing 
    3 │ 2012-01-01T00:10:00     23     75  missing  missing  missing  missing 
    4 │ 2012-01-01T00:15:00     23     75  missing  missing  missing  missing 
    5 │ 2012-01-01T00:20:00     23     75

In [ ]:
input_data(test)

Row,date,temp,rh,p_atm,Jp,Js,WS,WD,Qs,tau,cloudiness,pv,ah
,DateTime,Float64,Int64,Missing,Float64,Missing,Float64,String15,Missing,Float64,Missing,Float64,Float64
1,2012-01-01T00:00:00,3.7,77,missing,0.0,missing,1.3,北,missing,1.0e-50,missing,613.29,0.00378771
2,2012-01-01T01:00:00,3.9,74,missing,0.0,missing,1.3,北,missing,1.0e-50,missing,597.756,0.0036912
3,2012-01-01T02:00:00,3.7,76,missing,0.0,missing,1.9,北北西,missing,1.0e-50,missing,605.325,0.00373822
4,2012-01-01T03:00:00,4.2,74,missing,0.0,missing,1.0,北北西,missing,1.0e-50,missing,610.493,0.00377033
5,2012-01-01T04:00:00,5.0,71,missing,0.0,missing,1.6,北,missing,1.0e-50,missing,619.466,0.00382608
6,2012-01-01T05:00:00,4.7,73,missing,0.0,missing,1.1,北北西,missing,1.0e-50,missing,623.712,0.00385247
7,2012-01-01T06:00:00,4.7,71,missing,0.0,missing,0.8,東北東,missing,1.0e-50,missing,606.624,0.00374629
8,2012-01-01T07:00:00,4.4,75,missing,0.0,missing,1.3,北,missing,0.139735,missing,627.484,0.00387591
9,2012-01-01T08:00:00,5.2,72,missing,0.0,missing,1.1,北,missing,0.606535,missing,637.007,0.00393511
